# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_df = pd.read_csv("../weatherpy/city_weather.csv")
city_df

,Unnamed: 0,city,latitude,longitude,date,temperature,humidity,cloudiness,wind speed
0,0,Rikitea,-23.1203,-134.9692,1643609638,77.27,63,100,5.53
1,1,Dikson,73.5069,80.5464,1643609639,-3.37,94,100,22.30
2,2,Chicama,-7.8447,-79.1469,1643609639,64.94,85,96,3.36
3,3,Ushuaia,-54.8000,-68.3000,1643609639,42.46,75,100,8.05
4,4,Butaritari,3.0707,172.7902,1643609530,82.06,77,55,18.97
...,...,...,...,...,...,...,...,...,...
530,530,Tazmalt,36.3868,4.3989,1643609839,45.97,37,0,3.51
531,531,Turukhansk,65.8167,87.9833,1643609839,1.45,93,99,12.35
532,532,Flinders,-34.5833,150.8552,1643609840,82.74,71,28,4.00
533,533,Nouadhibou,20.9310,-17.0347,1643609840,64.38,72,100,19.57


In [4]:
g_key

'YAIzaSyB9bQ8N_A9baEchLGipW3m-7cYY6SXu168'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# locations = city_df[['latitude','longitude']]
# rating=city_df['humidity']


# figure_layout={'width'}
# # Plot Heatmap
# fig = gmaps.figure()

# # Create heat layer
# heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)


# # Add layer
# fig.add_layer(heat_layer)

# # Display figure
# fig

# Set dimensions for graph

#set location
locations = city_df[["latitude", "longitude"]].astype(float)
# Set the humidity
humidity = city_df["humidity"].astype(float)


figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [13]:
city_df=city_df.loc[((city_df["temperature"]<80) & (city_df["temperature"]>70))]
city_df=city_df.loc[city_df["wind speed"]<10]
city_df=city_df.loc[city_df["cloudiness"]==0]
city_df

,Unnamed: 0,city,latitude,longitude,date,temperature,humidity,cloudiness,wind speed
15,15,Pasni,25.2631,63.4710,1643609644,72.48,59,0,8.79
34,34,Cape Town,-33.9258,18.4232,1643609652,71.92,72,0,1.99
178,178,East London,-33.0153,27.9116,1643609626,76.14,83,0,5.75
256,256,Salalah,17.0151,54.0924,1643609534,75.29,60,0,2.30
323,323,Santa Fe,-31.6333,-60.7000,1643609505,75.02,57,0,2.93
398,398,Uthal,25.8072,66.6219,1643609790,74.53,56,0,4.23
401,401,Būrewāla,30.1667,72.6500,1643609701,71.69,18,0,3.56
456,456,Sur,22.5667,59.5289,1643609811,76.33,50,0,2.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df=pd.DataFrame(city_df)
hotel_df["Hotel Name"]=""
hotel_df=hotel_df.dropna()

In [19]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
#         cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
#         cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [20]:
hotel_df.head()

,Unnamed: 0,city,latitude,longitude,date,temperature,humidity,cloudiness,wind speed,Hotel Name
15,15,Pasni,25.2631,63.4710,1643609644,72.48,59,0,8.79,
34,34,Cape Town,-33.9258,18.4232,1643609652,71.92,72,0,1.99,
178,178,East London,-33.0153,27.9116,1643609626,76.14,83,0,5.75,
256,256,Salalah,17.0151,54.0924,1643609534,75.29,60,0,2.30,
323,323,Santa Fe,-31.6333,-60.7000,1643609505,75.02,57,0,2.93,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
